In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [50]:
data = np.loadtxt('../data/data.txt')

R = data[:, 0]  # Widerstand in Ohm
delta_t = data[:, 1]  # zeitlicher Abstand in S
U = data[:, 2]  # Spannung in V
I = data[:, 3] / 1000  # Stromstärke in A
M = 63.546  # molare Masse von Kupfer in g/mol
m = 342  # Masse der Probe in g


# Step 3: Perform a linear fit using np.polyfit (degree 1 for linear)
# coefficients = np.polyfit(x, y, 1)  # Returns [slope, intercept]

# # Step 4: Create a linear model with the fitted coefficients
# linear_fit = np.poly1d(coefficients)

# # Step 5: Plot the data and the linear fit
# plt.scatter(x, y, label='Data', color='blue')  # Original data points
# plt.plot(x, linear_fit(x), label='Linear Fit', color='red')  # Fitted line
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.legend()
# plt.show()

In [95]:
T = np.round(0.00134 * R**2 + 2.296 * R - 243.02)[1:]
E = (U * I * delta_t)[1:]
C_p = M/m * E/10

B = 137.8e9
V_0 = 7.092e-6
alpha = np.array([10.04,10.96,11.70,12.29,12.81,13.29,13.69,14.01,14.33,14.58,14.81,15.03,15.26,15.46,15.64,15.80,15.96,16.15,16.28])*10e-7

C_V = C_p - 9 * (T + 273.15) * V_0 * alpha**2 * B

print(9 * (T + 273.15) * V_0 * alpha**2 * B)
new_data = np.column_stack((T,E,C_p,C_V))
print(np.round(new_data,2))
# np.savetxt("../results/new_data.txt", new_data)

# # Convert to a DataFrame for better formatting
# df = pd.DataFrame(new_data, columns=["Temperature (K)", "Energy (J)", "$C_p$ (J/(mol*K))"])

# # Generate LaTeX table
# latex_table = df.to_latex(index=False, 
#                           float_format="%.2f", 
#                           header=["Temperature (K)", "Energy (J)", "$C_p$ (J/(mol*K))"])

# # Save the LaTeX table to a file
# with open("../results/table_new_data.tex", "w") as file:
#     file.write(latex_table)

# print("LaTeX table has been saved to 'table.tex'")




[0.08258682 0.108981   0.13623439 0.16360578 0.19217633 0.22238309
 0.25245522 0.28165911 0.31273417 0.34243837 0.37261925 0.40364097
 0.43657097 0.46911176 0.50161368 0.53388645 0.56715813 0.60368291
 0.63675221]
[[-180.    830.96   15.44   15.36]
 [-170.    927.58   17.24   17.13]
 [-160.    917.38   17.05   16.91]
 [-150.    940.83   17.48   17.32]
 [-140.    959.08   17.82   17.63]
 [-130.   1026.15   19.07   18.84]
 [-120.   1062.44   19.74   19.49]
 [-110.   1141.43   21.21   20.93]
 [-100.   1152.03   21.41   21.09]
 [ -90.   1113.17   20.68   20.34]
 [ -80.   1110.73   20.64   20.27]
 [ -70.   1084.14   20.14   19.74]
 [ -60.   1301.73   24.19   23.75]
 [ -50.   1711.84   31.81   31.34]
 [ -40.   1309.45   24.33   23.83]
 [ -30.   1331.94   24.75   24.21]
 [ -20.   1241.91   23.08   22.51]
 [ -10.   1186.71   22.05   21.45]
 [  -0.   1084.19   20.15   19.51]]
